In [1]:
import requests
import csv

In [2]:
response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?q=SPORTS&begin_date=20190101&api-key=bZaz1p3BiOatd4UTfTxYPo0R9B0cx0fB")

data = response.json() 
#print(data)
hits = data["response"]["meta"]["hits"]
max_page = int((hits / 10) - 1)
print(max_page)

303


In [19]:
def parse_articles(articles):
    '''
    This function takes in a response to the NYT api and parses
    the articles into a list of dictionaries
    '''
    news = []
    for i in articles['response']['docs']:
        dic = {}
        if i.get('id', 0):  
            dic['id'] = i['_id']
        if i.get('headline', 0):  
            dic['headline'] = i['headline']['main'].encode("utf8")
        if i.get('date', 0):  
            dic['date'] = i['pub_date'][0:10] # cutting time of day.
        if i.get('section', 0):  
            dic['section'] = i['section_name']
        if i['snippet'] is not None:
            dic['snippet'] = i['snippet'].encode("utf8")
        if i.get('source', 0):
            dic['source'] = i['source']
        #if i['source'] is not None:
            #dic['source'] = i['source']
        if i.get('type_of_material', 0):  
            dic['type'] = i['type_of_material']
        if i.get('web_url', 0):  
            dic['url'] = i['web_url']
        if i.get('word_count', 0):  
            dic['word_count'] = i['word_count']
        # locations
        locations = []
        for x in range(0,len(i['keywords'])):
            if 'glocations' in i['keywords'][x]['name']:
                locations.append(i['keywords'][x]['value'])
        dic['locations'] = locations
        # subject
        subjects = []
        for x in range(0,len(i['keywords'])):
            if 'subject' in i['keywords'][x]['name']:
                subjects.append(i['keywords'][x]['value'])
        dic['subjects'] = subjects   
        news.append(dic)
    return(news) 

In [27]:
#Nw4HZ3XGu0pCTuVFupaz1IQgzko2lyZ3
#bZaz1p3BiOatd4UTfTxYPo0R9B0cx0fB
import time
web_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?q=SPORTS&begin_date=20190101&api-key=Nw4HZ3XGu0pCTuVFupaz1IQgzko2lyZ3"
data_list = []
#time.sleep(80)
for i in range(180,200):
    time.sleep(15)
    reques_url = web_url + "&page=" + str(i)
    print(reques_url)
    response = requests.get(reques_url)
    #time.sleep(21)
    data = response.json() 
    if data['response']:
        new = parse_articles(data)
        data_list = data_list+new
   


keys = data_list[0].keys()
with open('ny_nfl_4Apr8.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data_list)    


https://api.nytimes.com/svc/search/v2/articlesearch.json?q=SPORTS&begin_date=20190101&api-key=Nw4HZ3XGu0pCTuVFupaz1IQgzko2lyZ3&page=180
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=SPORTS&begin_date=20190101&api-key=Nw4HZ3XGu0pCTuVFupaz1IQgzko2lyZ3&page=181
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=SPORTS&begin_date=20190101&api-key=Nw4HZ3XGu0pCTuVFupaz1IQgzko2lyZ3&page=182
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=SPORTS&begin_date=20190101&api-key=Nw4HZ3XGu0pCTuVFupaz1IQgzko2lyZ3&page=183
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=SPORTS&begin_date=20190101&api-key=Nw4HZ3XGu0pCTuVFupaz1IQgzko2lyZ3&page=184
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=SPORTS&begin_date=20190101&api-key=Nw4HZ3XGu0pCTuVFupaz1IQgzko2lyZ3&page=185
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=SPORTS&begin_date=20190101&api-key=Nw4HZ3XGu0pCTuVFupaz1IQgzko2lyZ3&page=186
https://api.nytimes.com/svc/search/v2/articlesea

In [ ]:
reques_url = web_url + "&page=" + str(4)
response = requests.get(reques_url)
data = response.json()

In [5]:
print(data)



{'fault': {'faultstring': 'Rate limit quota violation. Quota limit  exceeded. Identifier : 6fcc5f24-5343-4ead-aab2-31857630c76b', 'detail': {'errorcode': 'policies.ratelimit.QuotaViolation'}}}


In [ ]:
print(maxpage)